In [1]:
# Remove the unknown zip then create new labels


import pandas as pd
import numpy as np
import os
import datetime
import gc
from haversine import haversine
import json
os.getcwd()
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))


output_folder="/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/output_"+str(datetime.datetime.now().date())+"/"

try:
    os.stat(output_folder)
except:
    os.mkdir(output_folder)

In [2]:
df_previous_zip_by_store=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/output_by_store_zip_sales_cum_JL_2019-06-27.csv",dtype=str)
df_previous_zip_by_store.head(2)

,location_id,customer_zip_code,zip_validation,sales,trans,customer_id_hashed,cum_sales,sales_pctg_cum,revenue_flag,cum_trans,trans_pctg_cum,transactions_flag,transaction_dt_nunique,transaction_dt_max,transaction_dt_min
0,1,43232,Valid_Zip,602768.3400000014,16101.0,3245,602768.3400000014,0.21498328905757522,P,16101.0,0.26084632083724846,P,363,2019-06-22,2018-06-24
1,1,43207,Valid_Zip,348158.19999999925,6688.0,1798,950926.5400000006,0.339157353920313,P,30159.0,0.4885947574765901,P,363,2019-06-22,2018-06-24


In [3]:
df_previous_zip_by_store['transaction_dt_nunique']=df_previous_zip_by_store['transaction_dt_nunique'].astype(int)
df_previous_zip_by_store['sales']=df_previous_zip_by_store['sales'].astype(float)
df_previous_zip_by_store['trans']=df_previous_zip_by_store['trans'].astype(float)
df_previous_zip_by_store['customer_id_hashed']=df_previous_zip_by_store['customer_id_hashed'].astype(int)
del df_previous_zip_by_store['cum_sales']
del df_previous_zip_by_store['sales_pctg_cum']
del df_previous_zip_by_store['cum_trans']
del df_previous_zip_by_store['trans_pctg_cum']
del df_previous_zip_by_store['revenue_flag']
del df_previous_zip_by_store['transactions_flag']

df_stores_in_2_months_6990=df_previous_zip_by_store[(df_previous_zip_by_store['transaction_dt_nunique']<=60) | (df_previous_zip_by_store['location_id']=="6990")]
df_stores_more_than_2_months=df_previous_zip_by_store[(df_previous_zip_by_store['transaction_dt_nunique']>60) & (df_previous_zip_by_store['location_id']!="6990")]


In [4]:
df_stores_more_than_2_months_valid=df_stores_more_than_2_months[df_stores_more_than_2_months['zip_validation']=="Valid_Zip"]
df_stores_more_than_2_months_unknown=df_stores_more_than_2_months[df_stores_more_than_2_months['zip_validation']!="Valid_Zip"]

df_stores_more_than_2_months_unknown['zip_validation'].unique()


array(['Unknown_Zip'], dtype=object)

In [5]:
df_output=pd.DataFrame()

for store,df_store in df_stores_more_than_2_months_valid.groupby("location_id"):
    total_valid_sales=df_store['sales'].sum()
    total_valid_trans=df_store['trans'].sum()
    
    df_store=df_store.sort_values("sales",ascending=False)
    df_store['cumulative_sales']=df_store['sales'].cumsum()
    df_store['cumu_pctg_sales']=df_store['cumulative_sales']/total_valid_sales
    df_store=df_store.reset_index()
    del df_store['index']
    
    df_store_1st=df_store.iloc[0,].to_frame().T
    df_store_1st['revenue_flag']="P"
    
    df_store_others=df_store.iloc[1:,]
    df_store_others['revenue_flag']=np.where(df_store_others['cumu_pctg_sales']>=0.8,"T",
                                  np.where(df_store_others['cumu_pctg_sales']>=0.6,"S","P"))
    df_store=df_store_1st.append(df_store_others)
    ######### 
    df_store=df_store.sort_values("trans",ascending=False)
    df_store['cumulative_trans']=df_store['trans'].cumsum()
    df_store['cumu_pctg_trans']=df_store['cumulative_trans']/total_valid_trans
    df_store=df_store.reset_index()
    del df_store['index']
    
    df_store_1st=df_store.iloc[0,].to_frame().T
    df_store_1st['trans_flag']="P"
    
    df_store_others=df_store.iloc[1:,]
    df_store_others['trans_flag']=np.where(df_store_others['cumu_pctg_trans']>=0.8,"T",
                                  np.where(df_store_others['cumu_pctg_trans']>=0.6,"S","P"))
    df_store=df_store_1st.append(df_store_others)

    df_output=df_output.append(df_store)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
func_agg={'customer_zip_code':"nunique",'sales':"sum",'trans':"sum","customer_id_hashed":"sum"}

df_matrics_data=df_output.groupby(['revenue_flag','trans_flag'])['customer_zip_code','sales','trans','customer_id_hashed'].agg(func_agg).reset_index()
df_matrics_data.head(2)

,revenue_flag,trans_flag,customer_zip_code,sales,trans,customer_id_hashed
0,P,P,4849,1.495196e+09,38638183.0,9127678
1,P,S,1271,1.385980e+08,3098561.0,910600


In [7]:
writer=pd.ExcelWriter(output_folder+"Excel_BL_new_ta_matrix_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")


In [8]:
for col in df_matrics_data.columns.tolist()[2:]:
    df=df_matrics_data[['revenue_flag','trans_flag',col]]
    globals()["df_all_"+col]=df.pivot(index="revenue_flag",columns="trans_flag",values=col).fillna(0)
    globals()["df_all_"+col].to_excel(writer,col)

In [9]:
df_matrics_data_store=df_output.groupby(['location_id','revenue_flag','trans_flag'])['customer_zip_code','sales','trans','customer_id_hashed'].agg(func_agg).reset_index()


for col in df_matrics_data.columns.tolist()[2:]:
    globals()['df_every_store_'+col]=pd.DataFrame()
    df=df_matrics_data_store[['location_id','revenue_flag','trans_flag',col]]
    for store,df_store in df.groupby("location_id"):
        del df_store['location_id']
        locals()["df_store_"+col]=df_store.pivot(index="revenue_flag",columns="trans_flag",values=col).fillna(0)
        locals()["df_store_"+col]['location_id']=store
        globals()['df_every_store_'+col]=globals()['df_every_store_'+col].append(locals()["df_store_"+col])
    globals()['df_every_store_'+col].to_excel(writer,"by_store_"+col)
    

In [10]:
writer.save()

In [11]:
# df_output.to_csv(output_folder+"BL_new_ta_by_store_zip_60_days_valid_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
df_stores_more_than_2_months_unknown.to_csv(output_folder+"BL_new_ta_by_store_zip_60_days_unknown_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
df_stores_in_2_months_6990.to_csv(output_folder+"BL_new_ta_by_store_zip_lower60_days_6990_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [12]:
df_summary=df_output.groupby(['location_id','revenue_flag','trans_flag','transaction_dt_nunique','transaction_dt_min','transaction_dt_max'])['customer_zip_code'].count().to_frame().reset_index()
df_summary.to_csv(output_folder+"BL_summary_zip_count_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [13]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190701-134908-815.txt",
                         dtype=str,sep="|",usecols=['location_id','latitude_meas','longitude_meas'])
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
df_output=pd.merge(df_output,store_list,on="location_id",how="left")

df_output_closed=df_output[pd.isnull(df_output['latitude_meas'])]
df_output=df_output[pd.notnull(df_output['latitude_meas'])]
df_output.to_csv(output_folder+"BL_new_ta_by_store_zip_60_days_valid_open_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
df_output_closed.to_csv(output_folder+"BL_new_ta_by_store_zip_60_days_valid_closed_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
del df_output_closed


In [14]:
def get_dist_in_df(zip_cd,store_lat,store_long):
    try:
        dist=haversine(zip_centers[zip_cd],(store_lat,store_long),miles=True)
    except:
        dist=np.nan
    return dist
df_output['dist_miles']=df_output.apply(lambda x: get_dist_in_df(x['customer_zip_code'],x['latitude_meas'],x['longitude_meas']),axis=1)


In [15]:
df_output_PS_50_miles=df_output[(df_output['trans_flag'].isin(['P','S'])) &(df_output['dist_miles']<=50)]
print(df_output_PS_50_miles.shape)
df_output_PS_50_miles.groupby(['location_id'])['customer_zip_code'].count().to_frame().reset_index().sort_values("customer_zip_code",ascending=False).head(10)


(22209, 18)


,location_id,customer_zip_code
605,318,92
1338,548,91
1147,5197,91
1281,5332,80
907,4566,78
168,1369,63
242,1501,61
407,1789,61
732,4195,61
1083,513,60


In [16]:
df_output_PS_50_miles['dist_miles'].max()

49.994776679479415

In [17]:
df_output_PS_50_miles.head(2)

,location_id,customer_zip_code,zip_validation,sales,trans,customer_id_hashed,transaction_dt_nunique,transaction_dt_max,transaction_dt_min,cumulative_sales,cumu_pctg_sales,revenue_flag,cumulative_trans,cumu_pctg_trans,trans_flag,longitude_meas,latitude_meas,dist_miles
0,1,43232,Valid_Zip,602768,16101,3245,363,2019-06-22,2018-06-24,602768,0.236332,P,16101,0.278853,P,-82.914789,39.913636,2.717654
1,1,43209,Valid_Zip,270028,7370,1340,363,2019-06-22,2018-06-24,1.22095e+06,0.47871,P,23471,0.406495,P,-82.914789,39.913636,3.095917


# Detecting overlaps between stores

In [18]:
df_store_zip_both=df_output_PS_50_miles[['location_id','trans_flag','customer_zip_code','latitude_meas','longitude_meas']].rename(columns={"customer_zip_code":"zip_list_P_and_S"}).drop_duplicates()
df_store_zip_both=df_store_zip_both.groupby(["location_id",'latitude_meas','longitude_meas'])['zip_list_P_and_S'].apply(set).to_frame().reset_index()
df_store_zip_both['zip_list_P_and_S']=df_store_zip_both['zip_list_P_and_S'].apply(lambda x: list(x))

df_store_zip_P=df_output_PS_50_miles[df_output_PS_50_miles['trans_flag']=="P"]
df_store_zip_P=df_store_zip_P[['location_id','trans_flag','customer_zip_code','latitude_meas','longitude_meas']].rename(columns={"customer_zip_code":"zip_list_P"}).drop_duplicates()
df_store_zip_P=df_store_zip_P.groupby(["location_id",'latitude_meas','longitude_meas'])['zip_list_P'].apply(set).to_frame().reset_index()
df_store_zip_P['zip_list_P']=df_store_zip_P['zip_list_P'].apply(lambda x: list(x))

df_store_zip_S=df_output_PS_50_miles[df_output_PS_50_miles['trans_flag']=="S"]
df_store_zip_S=df_store_zip_S[['location_id','trans_flag','customer_zip_code','latitude_meas','longitude_meas']].rename(columns={"customer_zip_code":"zip_list_S"}).drop_duplicates()
df_store_zip_S=df_store_zip_S.groupby(["location_id",'latitude_meas','longitude_meas'])['zip_list_S'].apply(set).to_frame().reset_index()
df_store_zip_S['zip_list_S']=df_store_zip_S['zip_list_S'].apply(lambda x: list(x))

df_store_zip=pd.merge(df_store_zip_both,df_store_zip_P,on=["location_id",'latitude_meas','longitude_meas'],how="outer")
df_store_zip=pd.merge(df_store_zip,df_store_zip_S,on=["location_id",'latitude_meas','longitude_meas'],how="outer")


In [19]:
df_store_zip_0=df_store_zip.copy()

df_store_zip.columns=["store_A_"+x for x in df_store_zip.columns.tolist()]
df_store_zip_0.columns=["store_B_"+x for x in df_store_zip_0.columns.tolist()]

df_store_zip['key']=1
df_store_zip_0['key']=1

df_store_zip=pd.merge(df_store_zip,df_store_zip_0,on="key",how="outer")
del df_store_zip_0


df_store_zip=df_store_zip[df_store_zip['store_A_location_id']!=df_store_zip['store_B_location_id']]
df_store_zip=df_store_zip.reset_index()
del df_store_zip['index']
del df_store_zip['key']

gc.collect()

105

In [20]:
def find_intersection(list_1,list_2):
    return list(set(list_1).intersection(set(list_2)))
def dist_miles(lat_1,log_1,lat_2,log_2):
    return haversine((lat_1,log_1),(lat_2,log_2),miles=True)

In [21]:
df_store_zip['intersection_both_P_S']=df_store_zip.apply(lambda x: find_intersection(x['store_A_zip_list_P_and_S'],x['store_B_zip_list_P_and_S']),axis=1)

df_store_zip['count_of_overlap_zips']=df_store_zip['intersection_both_P_S'].apply(lambda x: len(x))
df_store_zip['count_of_A_zips']=df_store_zip['store_A_zip_list_P_and_S'].apply(lambda x: len(x))
df_store_zip['count_of_B_zips']=df_store_zip['store_B_zip_list_P_and_S'].apply(lambda x: len(x))
df_store_zip['total_zips']=df_store_zip['count_of_A_zips']+df_store_zip['count_of_B_zips']-df_store_zip['count_of_overlap_zips']

df_store_zip['overlap_pctg']=df_store_zip['count_of_overlap_zips']/df_store_zip['total_zips']
df_store_zip=df_store_zip[df_store_zip['count_of_overlap_zips']>0]
df_store_zip['dist_between_stores']=df_store_zip.apply(lambda x: dist_miles(x['store_A_latitude_meas'],x['store_A_longitude_meas'],x['store_B_latitude_meas'],x['store_B_longitude_meas']),axis=1)
df_store_zip=df_store_zip.reset_index()
del df_store_zip['index']

In [22]:
df_store_zip.to_csv(output_folder+"BL_zip_overlap_between_stores_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [23]:
def store_pair_func(store_A,store_B):
    return sorted([store_A,store_B])
df_store_zip_unique_pairs=df_store_zip.copy()
df_store_zip_unique_pairs['store_pair']=df_store_zip_unique_pairs.apply(lambda x: store_pair_func(x['store_A_location_id'],x['store_B_location_id']),axis=1)
df_store_zip_unique_pairs['store_pair_str']=df_store_zip_unique_pairs['store_pair'].astype(str)
df_store_zip_unique_pairs=df_store_zip_unique_pairs.sort_values(['store_pair_str','store_A_location_id','store_B_location_id'])
df_store_zip_unique_pairs=df_store_zip_unique_pairs.drop_duplicates("store_pair_str")
df_store_zip_unique_pairs.to_csv(output_folder+"BL_zip_overlap_between_stores_unique_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [24]:
df_store_zip_unique_pairs['overlap_group']=np.where(df_store_zip_unique_pairs['overlap_pctg']>0.2,"20%+",
                                                   np.where(df_store_zip_unique_pairs['overlap_pctg']>0.15,"15%-20%",
                                                           np.where(df_store_zip_unique_pairs['overlap_pctg']>0.1,"10%-15%",
                                                                   np.where(df_store_zip_unique_pairs['overlap_pctg']>0.05,'5%-10%',
                                                                           np.where(df_store_zip_unique_pairs['overlap_pctg']>0.03,'3%-5%',"<3%")
                                                                           )
                                                                   )
                                                           )
                                                   )

df_store_zip_unique_pairs['dist_group']=np.where(df_store_zip_unique_pairs['dist_between_stores']>28,"28+ miles",
                                                   np.where(df_store_zip_unique_pairs['dist_between_stores']>20,"20-28 miles",
                                                           np.where(df_store_zip_unique_pairs['dist_between_stores']>10,"10-20 miles","0-10 miles")
                                                           )
                                                   )
df_summary_overlap_dist_group_pair_count=df_store_zip_unique_pairs.groupby(['overlap_group','dist_group'])['store_pair_str'].count().to_frame().reset_index().rename(columns={"store_pair_str":"pairs_count"})
df_summary_overlap_dist_group_unique_stores=df_store_zip_unique_pairs.groupby(['overlap_group','dist_group'])['store_pair'].sum().to_frame().reset_index().rename(columns={"store_pair":"stores_involved"})

df_store_zip_unique_pairs['zips_P_S_50_both_stores']=df_store_zip_unique_pairs['store_B_zip_list_P_and_S']+df_store_zip_unique_pairs['store_A_zip_list_P_and_S']
df_store_zip_unique_pairs['zips_P_S_50_both_stores']=df_store_zip_unique_pairs['zips_P_S_50_both_stores'].apply(lambda x: list(set(x)))
df_summary_overlap_dist_group_unique_zips=df_store_zip_unique_pairs.groupby(['overlap_group','dist_group'])['zips_P_S_50_both_stores'].sum().to_frame().reset_index().rename(columns={"zips_P_S_50_both_stores":"zips_involved"})



In [25]:
df_summary_overlap_dist_group_unique_stores.shape

(24, 3)

In [26]:
sort_dict={"<3%":1,"3%-5%":2,"5%-10%":3,"10%-15%":4,"15%-20%":5,"20%+":6}

In [27]:
df_summary_pivot_pair=df_summary_overlap_dist_group_pair_count.pivot_table(index="overlap_group",columns='dist_group',values="pairs_count").reset_index()
df_summary_pivot_pair['sort']=df_summary_pivot_pair['overlap_group'].apply(lambda x: sort_dict[x])
df_summary_pivot_pair=df_summary_pivot_pair.sort_values("sort").reset_index()
del df_summary_pivot_pair['sort']
del df_summary_pivot_pair['index']


In [28]:
df_summary_overlap_dist_group_unique_stores['stores_involved']=df_summary_overlap_dist_group_unique_stores['stores_involved'].apply(lambda x: set(x))
df_summary_overlap_dist_group_unique_stores['store_count_involved']=df_summary_overlap_dist_group_unique_stores['stores_involved'].apply(lambda x: len(x))

df_summary_pivot_store=df_summary_overlap_dist_group_unique_stores.pivot_table(index="overlap_group",columns='dist_group',values="store_count_involved").reset_index()
df_summary_pivot_store['sort']=df_summary_pivot_store['overlap_group'].apply(lambda x: sort_dict[x])
df_summary_pivot_store=df_summary_pivot_store.sort_values("sort").reset_index()
del df_summary_pivot_store['sort']
del df_summary_pivot_store['index']


In [29]:
df_summary_overlap_dist_group_unique_zips['zips_involved']=df_summary_overlap_dist_group_unique_zips['zips_involved'].apply(lambda x: set(x))
df_summary_overlap_dist_group_unique_zips['zip_count_involved']=df_summary_overlap_dist_group_unique_zips['zips_involved'].apply(lambda x: len(x))

df_summary_pivot_zip=df_summary_overlap_dist_group_unique_zips.pivot_table(index="overlap_group",columns='dist_group',values="zip_count_involved").reset_index()
df_summary_pivot_zip['sort']=df_summary_pivot_zip['overlap_group'].apply(lambda x: sort_dict[x])
df_summary_pivot_zip=df_summary_pivot_zip.sort_values("sort").reset_index()
del df_summary_pivot_zip['sort']
del df_summary_pivot_zip['index']


In [30]:
df_store_zip_unique_pairs['store_pair'][0]

['1', '1379']

In [31]:
dist_group=df_store_zip_unique_pairs.groupby(['dist_group'])['store_pair'].sum().to_frame().reset_index().rename(columns={"store_pair":"unique_store_list"})
dist_group['unique_store_list']=dist_group['unique_store_list'].apply(lambda x: list(set(x)))
dist_group['unique_store_count']=dist_group['unique_store_list'].apply(lambda x: len(x))

dist_group_pair_count=df_store_zip_unique_pairs.groupby(['dist_group'])['store_pair'].count().to_frame().reset_index().rename(columns={"store_pair":"pair_counts"})

dist_group=pd.merge(dist_group,dist_group_pair_count,on="dist_group")

In [32]:
store_list_July=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190701-134908-815.txt",dtype=str,sep="|")
store_list_July=store_list_July[['location_id','address_line_1','address_line_2','city_nm','state_nm','zip_cd','latitude_meas','longitude_meas']]
store_list_July['latitude_meas']=store_list_July['latitude_meas'].astype(float)
store_list_July['longitude_meas']=store_list_July['longitude_meas'].astype(float)
store_list_July=store_list_July[store_list_July['location_id']!="6990"]
store_list_July=store_list_July[store_list_July['location_id']!="145"]

store_lat_dict=store_list_July.set_index("location_id").to_dict()['latitude_meas']
store_long_dict=store_list_July.set_index("location_id").to_dict()['longitude_meas']


In [33]:
df_all_P_zips_by_store=df_output[df_output['trans_flag']=="P"].groupby(['location_id'])['customer_zip_code'].apply(list).to_frame().reset_index().rename(columns={"customer_zip_code":"all_P_zips"})
df_all_S_zips_by_store=df_output[df_output['trans_flag']=="S"].groupby(['location_id'])['customer_zip_code'].apply(list).to_frame().reset_index().rename(columns={"customer_zip_code":"all_S_zips"})

df_50_miles_P_zips_by_store=df_output[(df_output['trans_flag']=="P") & (df_output['dist_miles']<=50)].groupby(['location_id'])['customer_zip_code'].apply(list).to_frame().reset_index().rename(columns={"customer_zip_code":"50_miles_P_zips"})
df_50_miles_S_zips_by_store=df_output[(df_output['trans_flag']=="S") & (df_output['dist_miles']<=50)].groupby(['location_id'])['customer_zip_code'].apply(list).to_frame().reset_index().rename(columns={"customer_zip_code":"50_miles_S_zips"})

df_P_S_zips_by_store=pd.merge(df_all_P_zips_by_store,df_all_S_zips_by_store,on="location_id",how="outer")
df_P_S_zips_by_store=pd.merge(df_P_S_zips_by_store,df_50_miles_P_zips_by_store,on="location_id",how="outer")
df_P_S_zips_by_store=pd.merge(df_P_S_zips_by_store,df_50_miles_S_zips_by_store,on="location_id",how="outer")
df_P_S_zips_by_store=df_P_S_zips_by_store.fillna("[]")
df_P_S_zips_by_store['all_P_zips']=df_P_S_zips_by_store['all_P_zips'].astype(str).apply(lambda x: eval(x))
df_P_S_zips_by_store['all_S_zips']=df_P_S_zips_by_store['all_S_zips'].astype(str).apply(lambda x: eval(x))
df_P_S_zips_by_store['50_miles_P_zips']=df_P_S_zips_by_store['50_miles_P_zips'].astype(str).apply(lambda x: eval(x))
df_P_S_zips_by_store['50_miles_S_zips']=df_P_S_zips_by_store['50_miles_S_zips'].astype(str).apply(lambda x: eval(x))


In [34]:
df_P_S_zips_by_store[df_P_S_zips_by_store['all_S_zips'].apply(lambda x: x==[])]

,location_id,all_P_zips,all_S_zips,50_miles_P_zips,50_miles_S_zips
258,1531,[42431],[],[42431],[]
492,190,[42420],[],[42420],[]
542,1983,[88101],[],[88101],[]
779,4301,[95076],[],[95076],[]
813,4432,[93060],[],[93060],[]
949,4611,[93555],[],[93555],[]
1000,4665,[93230],[],[93230],[]


In [35]:
# Keep this table and remove all of the stores for the bottom group store list
dist_group

,dist_group,unique_store_list,unique_store_count,pair_counts
0,0-10 miles,"[541, 422, 4220, 4436, 4131, 4516, 1007, 4336,...",742,889
1,10-20 miles,"[1958, 422, 4220, 4436, 4131, 4516, 1007, 555,...",853,1282
2,20-28 miles,"[4220, 4436, 256, 4336, 1216, 1255, 1911, 4652...",597,579
3,28+ miles,"[1958, 541, 4220, 555, 256, 4336, 1216, 1255, ...",596,823


In [36]:

def dist_of_2_str_stores(pair_stores_str):
    pair_stores_str=eval(pair_stores_str)
    store_a=pair_stores_str[0]
    store_b=pair_stores_str[1]
    dist=haversine([store_lat_dict[store_a],store_long_dict[store_a]],[store_lat_dict[store_b],store_long_dict[store_b]],miles=True)
    return dist

# Only used in below loop
def find_nearest_store(store_id):
    shortest_dist=99999
    nearest_store=np.nan
    for i_store in store_list_July['location_id'].tolist():
        if i_store!=store_id:
            dist=haversine([store_lat_dict[store_id],store_long_dict[store_id]],[store_lat_dict[i_store],store_long_dict[i_store]],miles=True)
            if dist<shortest_dist:
                shortest_dist=dist
                nearest_store=i_store
    return nearest_store, shortest_dist

def find_overlaped_stores(store_id):
    df=df_store_zip_unique_pairs[df_store_zip_unique_pairs['store_pair'].apply(lambda x: store_id in x)]
    df=df.sort_values('overlap_pctg',ascending=True).reset_index()
    min_overlap_pctg=df.loc[0,'overlap_pctg']
    min_overlap_store_pair_str=df.loc[0,'store_pair_str']
    
    max_overlap_pctg=df.loc[len(df)-1,'overlap_pctg']
    max_overlap_store_pair_str=df.loc[len(df)-1,'store_pair_str']
    
    return min_overlap_pctg,min_overlap_store_pair_str,max_overlap_pctg,max_overlap_store_pair_str

In [38]:
nearest_store_dist_criteria_dict={"0-10 miles":0,"10-20 miles":10,"20-28 miles":20,"28+ miles":28}

writer=pd.ExcelWriter(output_folder+"BL_new_TA_pair_dist_table_summary_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
dist_group.to_excel(writer,"pairs_and_stores_by_pair_dist",index=False)

ordered_cols=['dist_group','location_id','address_line_1','address_line_2','city_nm','state_nm','zip_cd','latitude_meas','longitude_meas',
              'all_P_zips','all_S_zips','All_P&S_zips','50_miles_P_zips','50_miles_S_zips','All_P&S_zips_50_miles',
              'nearest_store','dist_to_nearest_store',
              'max_overlap_store','max_overlap_pctg','dist_max_overlap_pairs',
              'min_overlap_pctg','min_overlap_store','dist_min_overlap_pairs']

all_paired_group_store_df=pd.DataFrame(columns=ordered_cols)
all_single_store_df=pd.DataFrame(columns=['nearest_store_dist_criteria']+ordered_cols)
all_high_overlap_store_df=pd.DataFrame(columns=ordered_cols)

for dist,group in dist_group.groupby("dist_group"):
    store_list_in_dist_group=group['unique_store_list'].values[0]
    df_store_zip_unique_pairs_removed=df_store_zip_unique_pairs[(~df_store_zip_unique_pairs['store_A_location_id'].isin(store_list_in_dist_group)) &\
                                                                 (~df_store_zip_unique_pairs['store_B_location_id'].isin(store_list_in_dist_group))]

    df_removed_pairs_rows=df_store_zip_unique_pairs[df_store_zip_unique_pairs['dist_group']==dist]
    df_store_zip_unique_pairs_removed_stores=df_store_zip_unique_pairs[(df_store_zip_unique_pairs['store_A_location_id'].isin(store_list_in_dist_group)) |\
                                                                 (df_store_zip_unique_pairs['store_B_location_id'].isin(store_list_in_dist_group))]
    
    df_removed_stores_1=pd.DataFrame({"location_id":store_list_in_dist_group})
    df_removed_stores_1=pd.merge(df_removed_stores_1,store_list_July,on="location_id",how="left")
    df_removed_stores_1=pd.merge(df_removed_stores_1,df_P_S_zips_by_store,on="location_id",how="left")
    
    df_removed_stores_1['nearest_store']=df_removed_stores_1['location_id'].apply(lambda x: find_nearest_store(x)[0])
    df_removed_stores_1['dist_to_nearest_store']=df_removed_stores_1['location_id'].apply(lambda x: find_nearest_store(x)[1])

    df_removed_stores_1['min_overlap_pctg']=df_removed_stores_1['location_id'].apply(lambda x: find_overlaped_stores(x)[0])
    df_removed_stores_1['min_overlap_store']=df_removed_stores_1['location_id'].apply(lambda x: find_overlaped_stores(x)[1])
    df_removed_stores_1['max_overlap_pctg']=df_removed_stores_1['location_id'].apply(lambda x: find_overlaped_stores(x)[2])
    df_removed_stores_1['max_overlap_store']=df_removed_stores_1['location_id'].apply(lambda x: find_overlaped_stores(x)[3])

    df_removed_stores_1['All_P&S_zips']=df_removed_stores_1['all_P_zips']+df_removed_stores_1['all_S_zips']
    df_removed_stores_1['All_P&S_zips_50_miles']=df_removed_stores_1['50_miles_P_zips']+df_removed_stores_1['50_miles_S_zips']

    df_removed_stores_1['dist_min_overlap_pairs']=df_removed_stores_1['min_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))
    df_removed_stores_1['dist_max_overlap_pairs']=df_removed_stores_1['max_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))
    df_removed_stores_1['dist_group']=dist
        
    df_removed_stores_1=df_removed_stores_1[ordered_cols]
    df_removed_stores_1=df_removed_stores_1.sort_values("dist_to_nearest_store",ascending=False)
    
    
    nearest_store_dist_criteria=nearest_store_dist_criteria_dict[dist]
    single_store_df=df_removed_stores_1[(df_removed_stores_1['dist_to_nearest_store']>nearest_store_dist_criteria) &\
                                        (df_removed_stores_1['max_overlap_pctg']<=0.15)]
    single_store_df['nearest_store_dist_criteria']=nearest_store_dist_criteria
    
    high_overlap_store_df=df_removed_stores_1[(df_removed_stores_1['dist_to_nearest_store']<=10) &\
                                        (df_removed_stores_1['max_overlap_pctg']>=0.5)]
       
    df_removed_stores_1.to_excel(writer,dist+"_all_stores",index=False)
    
    
    all_paired_group_store_df=all_paired_group_store_df.append(df_removed_stores_1)
    all_single_store_df=all_single_store_df.append(single_store_df)

    all_high_overlap_store_df=all_high_overlap_store_df.append(high_overlap_store_df)
    

def clean_list_col(df):
    for col in df.columns.tolist():
        if df[col].apply(lambda x: type(x)).unique()[0]==list:
            df[col]=df[col].astype(str)
    return df
            
all_single_store_df=clean_list_col(all_single_store_df)
all_paired_group_store_df=clean_list_col(all_paired_group_store_df)
all_high_overlap_store_df=clean_list_col(all_high_overlap_store_df)

all_single_store_df=all_single_store_df.drop_duplicates()
all_paired_group_store_df=all_paired_group_store_df.drop_duplicates()
all_high_overlap_store_df=all_high_overlap_store_df.drop_duplicates()

all_single_store_df=all_single_store_df[['nearest_store_dist_criteria']+ordered_cols]
all_single_store_df.to_excel(writer,"all_single_store_df",index=False)
all_paired_group_store_df.to_excel(writer,"all_paired_group_store_df",index=False)
all_high_overlap_store_df.to_excel(writer,"all_high_overlap_store_df",index=False)

all_unique_high_overlap_stores=all_high_overlap_store_df.copy()
del all_high_overlap_store_df['dist_group']
all_high_overlap_store_df=all_high_overlap_store_df.drop_duplicates()
all_high_overlap_store_df=all_high_overlap_store_df.sort_values(["state_nm",'city_nm','dist_to_nearest_store'])
all_high_overlap_store_df.to_excel(writer,"unique_all_high_overlap_store",index=False)

df_summary_high_overlap_store_st=all_high_overlap_store_df.groupby(all_high_overlap_store_df['state_nm'])['location_id'].nunique().to_frame().reset_index()
df_summary_high_overlap_store_st.to_excel(writer,"summary_high_overlap_st_count",index=False)
writer.save()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# All embeded in the loop above

In [41]:
'''
stores_to_removed_list=sorted(dist_group.loc[dist_group['dist_group']=="28+ miles",'unique_store_list'].values[0])
df_store_zip_unique_pairs_removed_once=df_store_zip_unique_pairs[(~df_store_zip_unique_pairs['store_A_location_id'].isin(stores_to_removed_list)) &\
                                                                 (~df_store_zip_unique_pairs['store_B_location_id'].isin(stores_to_removed_list))]
'''

(1018, 24)

In [45]:
'''
df_removed_pairs_rows=df_store_zip_unique_pairs[df_store_zip_unique_pairs['dist_group']=="28+ miles"]

df_store_zip_unique_pairs_removed_stores=df_store_zip_unique_pairs[(df_store_zip_unique_pairs['store_A_location_id'].isin(stores_to_removed_list)) |\
                                                                 (df_store_zip_unique_pairs['store_B_location_id'].isin(stores_to_removed_list))]
df_store_zip_unique_pairs_removed_stores
'''

,store_A_location_id,store_A_latitude_meas,store_A_longitude_meas,store_A_zip_list_P_and_S,store_A_zip_list_P,store_A_zip_list_S,store_B_location_id,store_B_latitude_meas,store_B_longitude_meas,store_B_zip_list_P_and_S,...,count_of_A_zips,count_of_B_zips,total_zips,overlap_pctg,dist_between_stores,store_pair,store_pair_str,overlap_group,dist_group,zips_P_S_50_both_stores
0,1,39.913636,-82.914789,"[43125, 43227, 43232, 43110, 43206, 43207, 432...","[43207, 43227, 43209, 43232]","[43125, 43110, 43206, 43213, 43205, 43068]",1379,40.141346,-82.985584,"[43081, 43229, 43334, 43212, 43017, 43230, 430...",...,10,38,43,0.116279,16.172938,"[1, 1379]","['1', '1379']",10%-15%,10-20 miles,"[43081, 43229, 43334, 43212, 43017, 43230, 432..."
1,1,39.913636,-82.914789,"[43125, 43227, 43232, 43110, 43206, 43207, 432...","[43207, 43227, 43209, 43232]","[43125, 43110, 43206, 43213, 43205, 43068]",1666,40.100960,-83.091519,"[43040, 43081, 43229, 43212, 43017, 43064, 432...",...,10,26,34,0.058824,15.968608,"[1, 1666]","['1', '1666']",5%-10%,10-20 miles,"[43040, 43081, 43229, 43212, 43017, 43064, 432..."


In [69]:
'''
df_removed_stores_1=pd.DataFrame({"location_id":stores_to_removed_list})
store_list_July=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190701-134908-815.txt",dtype=str,sep="|")
store_list_July=store_list_July[['location_id','address_line_1','address_line_2','city_nm','state_nm','zip_cd','latitude_meas','longitude_meas']]
store_list_July['latitude_meas']=store_list_July['latitude_meas'].astype(float)
store_list_July['longitude_meas']=store_list_July['longitude_meas'].astype(float)
store_list_July=store_list_July[store_list_July['location_id']!="6990"]
store_list_July=store_list_July[store_list_July['location_id']!="145"]

df_removed_stores_1=pd.merge(df_removed_stores_1,store_list_July,on="location_id",how="left")
df_removed_stores_1=pd.merge(df_removed_stores_1,df_P_S_zips_by_store,on="location_id",how="left")
''' 

In [78]:
'''
def find_nearest_store(store_id):
    shortest_dist=99999
    nearest_store=np.nan
    for i_store in store_list_July['location_id'].tolist():
        if i_store!=store_id:
            dist=haversine([store_lat_dict[store_id],store_long_dict[store_id]],[store_lat_dict[i_store],store_long_dict[i_store]],miles=True)
            if dist<shortest_dist:
                shortest_dist=dist
                nearest_store=i_store
    return nearest_store, shortest_dist
                
                
            
df_removed_stores_1['nearest_store']=df_removed_stores_1['location_id'].apply(lambda x: find_nearest_store(x)[0])
df_removed_stores_1['dist_to_nearest_store']=df_removed_stores_1['location_id'].apply(lambda x: find_nearest_store(x)[1])
'''

In [92]:
'''
def find_overlaped_stores(store_id):
    df=df_store_zip_unique_pairs[df_store_zip_unique_pairs['store_pair'].apply(lambda x: store_id in x)]
    df=df.sort_values('overlap_pctg',ascending=True).reset_index()
    min_overlap_pctg=df.loc[0,'overlap_pctg']
    min_overlap_store_pair_str=df.loc[0,'store_pair_str']
    
    max_overlap_pctg=df.loc[len(df)-1,'overlap_pctg']
    max_overlap_store_pair_str=df.loc[len(df)-1,'store_pair_str']
    
    return min_overlap_pctg,min_overlap_store_pair_str,max_overlap_pctg,max_overlap_store_pair_str
'''

In [97]:
'''
df_removed_stores_1['min_overlap_pctg']=df_removed_stores_1['location_id'].apply(lambda x: find_overlaped_stores(x)[0])
df_removed_stores_1['min_overlap_store']=df_removed_stores_1['location_id'].apply(lambda x: find_overlaped_stores(x)[1])
df_removed_stores_1['max_overlap_pctg']=df_removed_stores_1['location_id'].apply(lambda x: find_overlaped_stores(x)[2])
df_removed_stores_1['max_overlap_store']=df_removed_stores_1['location_id'].apply(lambda x: find_overlaped_stores(x)[3])
'''

In [101]:
'''
df_summary_overlap_dist_group_pair_count_1=df_store_zip_unique_pairs_removed_once.groupby(['overlap_group','dist_group'])['store_pair_str'].count().to_frame().reset_index().rename(columns={"store_pair_str":"pairs_count"})
df_summary_overlap_dist_group_unique_stores_1=df_store_zip_unique_pairs_removed_once.groupby(['overlap_group','dist_group'])['store_pair'].sum().to_frame().reset_index().rename(columns={"store_pair":"stores_involved"})

df_summary_overlap_dist_group_unique_zips_1=df_store_zip_unique_pairs_removed_once.groupby(['overlap_group','dist_group'])['zips_P_S_50_both_stores'].sum().to_frame().reset_index().rename(columns={"zips_P_S_50_both_stores":"zips_involved"})




df_summary_pivot_pair_1=df_summary_overlap_dist_group_pair_count_1.pivot_table(index="overlap_group",columns='dist_group',values="pairs_count").reset_index()
df_summary_pivot_pair_1['sort']=df_summary_pivot_pair_1['overlap_group'].apply(lambda x: sort_dict[x])
df_summary_pivot_pair_1=df_summary_pivot_pair_1.sort_values("sort").reset_index()
del df_summary_pivot_pair_1['sort']
del df_summary_pivot_pair_1['index']
##########
df_summary_overlap_dist_group_unique_stores_1['stores_involved']=df_summary_overlap_dist_group_unique_stores_1['stores_involved'].apply(lambda x: set(x))
df_summary_overlap_dist_group_unique_stores_1['store_count_involved']=df_summary_overlap_dist_group_unique_stores_1['stores_involved'].apply(lambda x: len(x))

df_summary_pivot_store_1=df_summary_overlap_dist_group_unique_stores_1.pivot_table(index="overlap_group",columns='dist_group',values="store_count_involved").reset_index()
df_summary_pivot_store_1['sort']=df_summary_pivot_store_1['overlap_group'].apply(lambda x: sort_dict[x])
df_summary_pivot_store_1=df_summary_pivot_store_1.sort_values("sort").reset_index()
del df_summary_pivot_store_1['sort']
del df_summary_pivot_store_1['index']
##########
df_summary_overlap_dist_group_unique_zips_1['zips_involved']=df_summary_overlap_dist_group_unique_zips_1['zips_involved'].apply(lambda x: set(x))
df_summary_overlap_dist_group_unique_zips_1['zip_count_involved']=df_summary_overlap_dist_group_unique_zips_1['zips_involved'].apply(lambda x: len(x))

df_summary_pivot_zip_1=df_summary_overlap_dist_group_unique_zips_1.pivot_table(index="overlap_group",columns='dist_group',values="zip_count_involved").reset_index()
df_summary_pivot_zip_1['sort']=df_summary_pivot_zip_1['overlap_group'].apply(lambda x: sort_dict[x])
df_summary_pivot_zip_1=df_summary_pivot_zip_1.sort_values("sort").reset_index()
del df_summary_pivot_zip_1['sort']
del df_summary_pivot_zip_1['index']
'''

In [110]:
'''
df_removed_stores_1['All_P&S_zips']=df_removed_stores_1['all_P_zips']+df_removed_stores_1['all_S_zips']
df_removed_stores_1['All_P&S_zips_50_miles']=df_removed_stores_1['50_miles_P_zips']+df_removed_stores_1['50_miles_S_zips']
'''

In [111]:
'''
def dist_of_2_str_stores(pair_stores_str):
    pair_stores_str=eval(pair_stores_str)
    store_a=pair_stores_str[0]
    store_b=pair_stores_str[1]
    dist=haversine([store_lat_dict[store_a],store_long_dict[store_a]],[store_lat_dict[store_b],store_long_dict[store_b]],miles=True)
    return dist
    
df_removed_stores_1['dist_min_overlap_pairs']=df_removed_stores_1['min_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))
df_removed_stores_1['dist_max_overlap_pairs']=df_removed_stores_1['max_overlap_store'].apply(lambda x: dist_of_2_str_stores(x))
'''

In [113]:
'''
writer=pd.ExcelWriter(output_folder+"Excel_BL_new_TA_pair_range_summary_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_summary_pivot_pair.to_excel(writer,"original_count_of_pairs",index=False)
df_summary_pivot_store.to_excel(writer,"original_count_of_unique_stores",index=False)
df_summary_pivot_zip.to_excel(writer,"original_count_of_unique_zips",index=False)

dist_group.to_excel(writer,"dist_group_store",index=False)
df_removed_pairs_rows.to_excel(writer,"df_removed_pairs_rows",index=False)
df_removed_stores_1.to_excel(writer,"removed_stores_28Plus",index=False)

df_summary_pivot_pair_1.to_excel(writer,"1stRemove_of_pairs",index=False)
df_summary_pivot_store_1.to_excel(writer,"1stRemove_of_unique_stores",index=False)
df_summary_pivot_zip_1.to_excel(writer,"1stRemove_of_unique_zips",index=False)

writer.save()
'''

# Aggregate into TA

In [21]:
# Use transaction labels
# P, S only to treat the same TA
# Too many Ss make a big TA
df_store_zip=df_output_PS_50_miles[['location_id','trans_flag','customer_zip_code']].rename(columns={"customer_zip_code":"zip_list"}).drop_duplicates()
print(df_store_zip['zip_list'].nunique())

df_store_zip=df_store_zip.groupby("location_id")['zip_list'].apply(set).to_frame().reset_index()
df_store_zip['zip_list']=df_store_zip['zip_list'].apply(lambda x: list(x))
df_store_zip.shape


12543


(1389, 2)

In [22]:
df_TA_zips=pd.DataFrame({"location_id":[df_store_zip['location_id'][0]]*len(df_store_zip['zip_list'][0]),
                         "zip_cd":df_store_zip['zip_list'][0],"TA":[1]*len(df_store_zip['zip_list'][0])},
                        index=[0]*len(df_store_zip['zip_list'][0]))

df_TA_zips.head(2)

,TA,location_id,zip_cd
0,1,1,43206
0,1,1,43207


In [23]:
TA_counter=1

for i in range(1,len(df_store_zip)):
    intersection_zip=list(set(df_store_zip.loc[i,'zip_list']).intersection(set(df_TA_zips['zip_cd'].unique().tolist())))
    if len(intersection_zip)==0:
        TA_counter+=1
        df_new_ta_store=pd.DataFrame({"location_id":[df_store_zip['location_id'][i]]*len(df_store_zip['zip_list'][i]),
                                      "zip_cd":df_store_zip['zip_list'][i],
                                      "TA":[TA_counter]*len(df_store_zip['zip_list'][i])},
                                     index=[i]*len(df_store_zip['zip_list'][i]))
        df_TA_zips=df_TA_zips.append(df_new_ta_store).drop_duplicates()
        
    else:
        df_intersection=df_TA_zips[df_TA_zips['zip_cd'].isin(intersection_zip)]
        group_df_intersection=df_intersection.groupby(['TA'])['zip_cd'].count().to_frame().reset_index().sort_values(['zip_cd'],ascending=False)
        selected_TA=group_df_intersection['TA'].tolist()[0] 
        
        df_TA_zips_0=df_TA_zips[~df_TA_zips['TA'].isin(set(group_df_intersection['TA']))]
        df_TA_zips_1=df_TA_zips[df_TA_zips['TA'].isin(group_df_intersection['TA'].tolist())]
        df_TA_zips_1['TA']=selected_TA
        df_overlaped_ta_store=pd.DataFrame({"location_id":[df_store_zip['location_id'][i]]*len(df_store_zip['zip_list'][i]),
                                            "zip_cd":df_store_zip['zip_list'][i],
                                            "TA":[selected_TA]*len(df_store_zip['zip_list'][i])},
                                           index=[i]*len(df_store_zip['zip_list'][i]))
        df_TA_zips=df_TA_zips_0.append(df_TA_zips_1).append(df_overlaped_ta_store).drop_duplicates()



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
rename_TA=df_TA_zips[['TA']].drop_duplicates()
rename_TA['new_TA_name']=[x+1 for x in range(len(rename_TA))]

df_TA_zips=pd.merge(df_TA_zips,rename_TA,on="TA",how="outer")
dict_TA_zips=df_TA_zips.set_index('zip_cd').to_dict()['new_TA_name']
del df_TA_zips['TA']


dict_TA_store=df_TA_zips.set_index('location_id').to_dict()['new_TA_name']
df_store_zip['TA']=df_store_zip['location_id'].apply(lambda x: dict_TA_store[x])
df_store_zip=df_store_zip.rename(columns={'zip_list':'PS_zip_list'})


In [25]:
df_store_zip.shape

(1389, 3)

In [31]:
df_store_zip.head(2)

,location_id,PS_zip_list,TA
0,1,"[43206, 43207, 43227, 43110, 43068, 43125, 432...",246
1,1001,"[37357, 38581, 37166, 37110]",245


In [33]:
df_TA_zips.head(2)

,location_id,zip_cd,new_TA_name
0,1021,38751,1
1,1021,38924,1


In [26]:
df_store_zip.groupby(['TA'])['location_id'].count().to_frame().reset_index().sort_values("location_id",ascending=False).head(10)





,TA,location_id
244,245,216
231,232,93
172,173,90
245,246,80
248,249,70
185,186,50
238,239,44
241,242,33
247,248,31
166,167,22


In [27]:
df_store_zip

,location_id,PS_zip_list,TA
0,1,"[43206, 43207, 43227, 43110, 43068, 43125, 432...",246
1,1001,"[37357, 38581, 37166, 37110]",245
2,1003,"[30120, 30171, 30103, 30184, 30101, 30121, 301...",245
3,1005,"[27295, 27292, 27299, 27107, 27360]",245
4,1006,"[28467, 28470, 29569, 29597, 29588, 29554, 295...",245
5,1007,"[66030, 66204, 66021, 66212, 66071, 66215, 662...",221
6,1009,"[70043, 70452, 70445, 70469, 70459, 70461, 704...",208
7,1011,"[30028, 30040, 30024, 30041, 30518, 30533, 305...",245
8,1012,"[29016, 29063, 29204, 29206, 29229, 29044, 291...",245
9,1013,"[27101, 27051, 27040, 27127, 27019, 27105, 271...",245


In [28]:
df_store_zip.shape

(1389, 3)

In [29]:
df_store_zip.head(2)

,location_id,PS_zip_list,TA
0,1,"[43206, 43207, 43227, 43110, 43068, 43125, 432...",246
1,1001,"[37357, 38581, 37166, 37110]",245


In [30]:
df_store_zip['TA'].nunique()

249

In [107]:
test_big_TA_stores=df_store_zip[df_store_zip['TA']==235]['location_id'].tolist()
# print(test_big_TA_stores)
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190701-134908-815.txt",dtype=str,sep="|")
store_list[store_list['location_id'].isin(test_big_TA_stores)]

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
100,804,NaN,1994-11-11,NaN,1280 N HENDERSON ST,NaN,GALESBURG,IL,61401-1578,-000000000090.385858,000000000040.963520,00000000021259.0000,00000000003426.0000
183,1289,NaN,1998-11-03,NaN,3320 AGENCY ST,NaN,BURLINGTON,IA,52601-1966,-000000000091.142883,000000000040.815678,00000000020039.0000,00000000004042.0000


In [108]:
store_list[store_list['location_id'].isin(test_big_TA_stores)].shape

(2, 13)